In [ ]:
# IMPORTS
%load_ext autoreload
%autoreload 2
import numpy as np
from scipy.fft import irfft, rfft, rfftfreq, fft, fftfreq, fftshift, ifft
import matplotlib.pyplot as plt
import matplotlib
from scipy.stats import norm
import pandas as pd
from scipy.special import wofz
from scipy.optimize import curve_fit
from scipy.signal import welch, get_window
%matplotlib widget 
from funcs_peak_fitting import *
from funcs_dsp import *
from funcs_df import load_df

In [ ]:
# Load dataframe
laptop = True
dfs_to_load = ["Curated Data"] # If this is empty, all are loaded
df = load_df(laptop=laptop, dfs_to_load=dfs_to_load)
# Crop to only wf
df = df[df['wf'].notna()]
# Crop to only species
df_human = df[df['species'].isin(["Human"])]
df_lizard = df[df['species'].isin(["Lizard", "Anolis"])]


In [ ]:
# Grab waveforms
h_idx = 0
l_idx = 0

wf_human = df_human.iloc[h_idx]['wf']
wf_lizard = df_lizard.iloc[l_idx]['wf']
fs_human = df_human.iloc[h_idx]['sr']
fs_lizard = df_lizard.iloc[l_idx]['sr']
if fs_human == fs_lizard:
    fs = fs_human
else:
    raise("Shouldn't these all have the same samplerate?")

In [ ]:
""" Get PSDs and plot SOAE data """
plt.close('all')
# Parameters
scaling = "density"
detrend = None # No detrending
win_type = 'boxcar'
nperseg = 32768
zpad = 2
nfft = nperseg*zpad
log = False
f_min = 500 # Set minimum frequency for crop

f_human, psd_human = welch(wf_human, scaling=scaling, fs=fs, window=win_type, nperseg=nperseg, nfft=nfft, detrend=detrend)
f_lizard, psd_lizard = welch(wf_lizard, scaling=scaling, fs=fs, window=win_type, nperseg=nperseg, nfft=nfft, detrend=detrend)


if np.array_equal(f_human, f_lizard):
    f = f_human
else:
    raise("Why aren't these the same?")

f_min_idx = np.argmin(np.abs(f - f_min)) # Convert frequency to index
f_max_idx = f_min_idx + 4096*zpad

# Crop frequencies
f_cropped = f[f_min_idx:f_max_idx]
psd_lizard = psd_lizard[f_min_idx:f_max_idx]
psd_human = psd_human[f_min_idx:f_max_idx]



for psd, species in zip([psd_human, psd_lizard], ["Human", "Lizard"]):
    plt.figure(figsize=(10, 5))
    if scaling == 'spectrum':
        label = "Power Spectrum"
        ylabel = "PS"
    elif scaling == 'density':
        label = "Power Spectral Density"
        ylabel = "PSD"
        
    if log:
        # Convert to log
        psd = 10 * np.log10(psd)
        ylabel += " (Log)"
    else:
        ylabel += " (Linear)"
    
    if detrend == "constant":
        label += " (Detrended)"
    elif detrend == False:
        label += " (Not Detrended)"
        
    label+=f": nperseg={nperseg}"
    plt.title(f"{species} PSD (Cropped)")
    plt.plot(f_cropped, psd, label="Proposed Crop")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel(ylabel)
    plt.legend()
    plt.show()
    

In [ ]:
# Define fitting function


def fit_peak(x, y, f0, y0, amp, gamma, sigma, f_hw=100, log=False, species=None):
    voigt, lorentz, gauss, lorentz_smeared = Voigt, Lorentzian, Gauss, Lorentzian_conv
    # Calculate min and max freq idx to crop
    f_min = f0 - f_hw
    f_max = f0 + f_hw
    f_min_idx = np.argmin(np.abs(x - f_min))
    f_max_idx = np.argmin(np.abs(x - f_max))
    x = x[f_min_idx:f_max_idx]
    y = y[f_min_idx:f_max_idx]
    
    # Set bounds (strangely, I have to let amp run negative to get a good fit...)
    bounds = (
        [f_min, 0, -1, 0, 0], # f0, y0, amp, gamma, sigma
        [f_max, np.inf, np.inf, np.inf, np.inf] 
    )
    
    # Set initial guess
    p0 = [f0, y0, amp, gamma, sigma]
    
    # Crop bounds/p0 for gauss and lorentz dirichlete less params
    bounds_lorentz = (bounds[0][:4], bounds[1][:4])
    p0_lorentz = p0[0:4]
    bounds_gauss = (bounds[0][:3] + [bounds[0][4]], bounds[1][:3] + [bounds[1][4]])
    p0_gauss = p0[0:3] + p0[4:]
    # bounds_dirichlet = (bounds[0][:3], bounds[1][:3])
    # p0_dirichlet = p0[0:3]
    
    
    
    lorentz_params, lorentz_cov = curve_fit(lorentz, x, y, bounds=bounds_lorentz, p0=p0_lorentz)
    gauss_params, gauss_cov = curve_fit(gauss, x, y, bounds=bounds_gauss, p0=p0_gauss)
    voigt_params, voigt_cov = curve_fit(voigt, x, y, bounds=bounds, p0=p0)
    # dirichlet_params, dirichlet_cov = curve_fit(dirichlet_mag, x, y, bounds=bounds_dirichlet, p0=p0_dirichlet)
    lorentz_smeared_params, lorentz_smeared_cov = curve_fit(lorentz_smeared, x, y, bounds=bounds_lorentz, p0=p0_lorentz)

    lorentz_fit = lorentz(x, *lorentz_params)
    gauss_fit = gauss(x, *gauss_params)
    voigt_fit = voigt(x, *voigt_params)
    # dirichlet_fit = dirichlet_mag(x, *dirichlet_params)
    lorentz_smeared_fit = lorentz_smeared(x, *lorentz_params)

    print("")
    
    # Print MSE Details
    scale1e = -22
    best_mse = np.inf
    best_type = None

    for type, fit in zip(["Lorentzian", "Gaussian", "Voigt",  "Lorentz Smeared"], [lorentz_fit, gauss_fit, voigt_fit, lorentz_smeared_fit]):
        mse = np.mean((fit - y)**2)
        if best_mse > mse:
            best_mse = mse
            best_type = type
        
        print(f"MSE (1e{scale1e}) of {type} = {mse*(10**-scale1e):.2f}")
    
    print(f"Best Fit = {best_type}")
    
    print("")
    
    # Print fitting parameters
    
    print("LORENTZIAN")
    print(f"f0 = {lorentz_params[0]:.2f}, y0 = {lorentz_params[1]*10**10:.2f}e-10, amp = {lorentz_params[2]*10**10:.2f}e-10, Gamma (HWHM) = {lorentz_params[-1]:.2f}")
    print("LORENTZIAN SMEARED")
    print(f"f0 = {lorentz_smeared_params[0]:.2f}, y0 = {lorentz_smeared_params[1]*10**10:.2f}e-10, amp = {lorentz_smeared_params[2]*10**10:.2f}e-10, Gamma (HWHM) = {lorentz_smeared_params[-1]:.2f}")
    print("GAUSSIAN")
    print(f"f0 = {gauss_params[0]:.2f}, y0 = {gauss_params[1]*10**10:.2f}e-10, amp = {gauss_params[2]*10**10:.2f}e-10, Sigma = {gauss_params[-1]:.2f}, HWHM = {get_gauss_hwhm(gauss_params[-1]):.2f}")
    print(f"VOIGT")
    print(f"f0 = {voigt_params[0]:.2f}, y0 = {voigt_params[1]*10**10:.2f}e-10, amp = {voigt_params[2]*10**10:.2f}e-10, Gamma = {voigt_params[-2]:.2f}, Sigma = {voigt_params[-1]:.2f}, HWHM = {get_voigt_hwhm(voigt_params[-2], voigt_params[-1]):.2f}")
    # print(f"SINC")
    # print(f"f0 = {dirichlet_params[0]:.2f}, y0 = {dirichlet_params[1]*10**10:.2f}e-10, amp = {dirichlet_params[2]*10**10:.2f}e-10")
    
    print("")
    

    ylabel = "PSD"
    if log:
        y = 10*np.log10(y)
        ylabel = ylabel + " (Log)"
        
    plt.close('all')
    plt.figure(figsize=(10, 5))
    plt.title(f"Peak Shape Fit")
    if species is not None:
        plt.title(f"Peak Shape Fit ({species})")
    plt.scatter(x, y, label="Original PSD", alpha=0.5, s=5, color='g', zorder=4)
    plt.plot(x, voigt_fit, label="Voigt Fit", color='c', zorder=1)
    plt.plot(x, lorentz_fit, label="Lorentzian Fit", color='b', zorder=2)
    plt.plot(x, gauss_fit, label="Gaussian Fit", color='m', zorder=3)
    plt.plot(x, lorentz_smeared_fit, label="Smeared Lorentzian Fit", color='y', zorder=5)
    plt.xlabel("Frequency (Hz)")
    plt.ylabel(ylabel)
    plt.legend()
    plt.show()

In [ ]:
"Lizard Peak Shape Fitting"
x = f_cropped
y = psd_lizard
log = False # Log scale
f_hw = 150 # Amount on either side to include
# Initial Guesses
f0 = 3710 # Peak center
y0 = 0 # Vertical shift
amp = 1e-8 # Peak max
gamma = 1 # FWHM of Lorentzian, or "amount" of Lorentzian in Voigt
sigma = 1 # Std of Gaussian, or "amount" of Gaussian in Voigt

print("Lizard SOAE Peak Fitting")
fit_peak(x, y, f0, y0, amp, gamma, sigma, f_hw=f_hw, log=False, species=None)

In [ ]:
"Human Peak Shape Fitting"
species = "Human"
x = f_cropped
y = psd_human
log = False # Log scale
f_hw = 5 # Amount on either side to include
# Initial Guesses
f0 = 2252 # Peak center (4372, 2252)
y0 = 0 # Vertical shift
amp = 3e-7 # Peak max
gamma = 1 # FWHM of Lorentzian, or "amount" of Lorentzian in Voigt
sigma = 1 # Std of Gaussian, or "amount" of Gaussian in Voigt


fit_peak(x, y, f0, y0, amp, gamma, sigma, f_hw=f_hw, log=log, species=species)